## DATA PREPROCESSING

In [45]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [46]:
df = pd.read_excel("category.xlsx",sheet_name= [1,4])
loc_to_cat = df[1]
loc_to_cat['category1'] = loc_to_cat['category1'].str.lower()
allUserData = df[4]
#userData = [df[3], df[4], df[5], df[6], df[7]]

#for i in userData[1:]:
#    allUserData = allUserData.append(i)
allUserData.head()

,user_id,weekday,date,start_time,end_date,end_time,duration,category,category_id,location_name,location_category,my_place,location_id,place_id
0,2,2,2020-06-29,16:00:00,2020-06-29,19:00:00,03:00:00,meeting,13,路易莎,cafe,NaN,4.0,NaN
1,2,2,2020-06-29,19:00:00,2020-06-29,20:00:00,01:00:00,food,1,全哥鵝肉小炒,restaurant,NaN,5.0,NaN
2,2,2,2020-06-29,20:00:00,2020-06-29,20:30:00,00:30:00,leisure,10,NaN,NaN,home,NaN,1.0
3,2,2,2020-06-29,20:30:00,2020-06-29,21:30:00,01:00:00,shopping,12,Miramar,Shopping mall,NaN,6.0,NaN
4,2,2,2020-06-29,21:30:00,2020-06-29,23:30:00,02:00:00,study,5,NaN,NaN,home,NaN,1.0


In [47]:
allUserData['location']= allUserData['location_id'].combine_first(allUserData['place_id'])
allUserData['location'].fillna("99", inplace = True)
allUserData = allUserData.drop(['date','end_date','end_time','location_name','location_category','my_place','location_id','place_id'], 1)

In [48]:
def get_hours(ts):
    a = ts.hour
    return a

def get_mins(ts):
    a = ts.minute
    return a

def get_duration(ts):
    a = ts.hour*60 + ts.minute
    return a 

#allUserData['start_mins'] = allUserData['start_time'].apply(get_mins)
allUserData['start_time'] = allUserData['start_time'].apply(get_duration)
allUserData['duration'] = allUserData['duration'].apply(get_duration)
allUserData = allUserData[['user_id','weekday','start_time','duration','location','category','category_id']]
allUserData.tail()

,user_id,weekday,start_time,duration,location,category,category_id
696,2,5,570,150,2,work,6
697,2,5,720,60,99,commute,17
698,2,5,780,360,3,meeting,13
699,2,5,1140,60,99,commute,17
700,2,5,1200,240,1,leisure,10


## Random Forest Model

In [78]:
train_df = allUserData.loc[0:599]
test_df = allUserData.loc[600:699]
print("Number of traing data is :",len(train_df))
print("Number of testing data is :",len(test_df))

Number of traing data is : 600
Number of testing data is : 100


In [79]:
# without diff
X_train = train_df.iloc[:, np.r_[0:4]].values
y_train = train_df.iloc[:, 6].values

X_test = test_df.iloc[:, np.r_[0:4]].values
y_test = test_df.iloc[:, 6].values

In [80]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0,
            warm_start=False)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=50, verbose=0, warm_start=False)

In [81]:
clf.score(X_test, y_test)

0.74

In [82]:
imp = clf.feature_importances_
imp

array([0.        , 0.10510417, 0.43900302, 0.45589281])

In [83]:
predictions = clf.predict(X_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           1       0.60      0.60      0.60        10
           3       1.00      1.00      1.00         8
           6       0.60      0.60      0.60         5
           9       1.00      1.00      1.00         2
          10       0.73      0.80      0.76        20
          11       0.00      0.00      0.00         3
          12       0.50      0.14      0.22         7
          13       1.00      0.67      0.80         3
          16       0.00      0.00      0.00         2
          17       0.73      0.95      0.83        38
          18       0.00      0.00      0.00         2

   micro avg       0.74      0.74      0.74       100
   macro avg       0.56      0.52      0.53       100
weighted avg       0.68      0.74      0.70       100



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [76]:
print(confusion_matrix(y_test,predictions))

[[ 6  0  0  0  2  0  0  0  0  2  0]
 [ 0  8  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  1  0  1  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0]
 [ 2  0  1  0 16  0  0  0  0  1  0]
 [ 0  0  0  0  1  0  0  0  0  2  0]
 [ 0  0  0  0  1  0  1  0  0  5  0]
 [ 0  0  1  0  0  0  0  2  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  1  0  0  0  0 37  0]
 [ 0  0  0  0  0  0  0  0  0  2  0]]


In [77]:
predictions

array([17,  9, 17, 17, 17, 10,  3, 10, 17, 10, 17, 17, 13, 17,  1, 17, 17,
       17, 10, 17, 10,  3, 10, 17,  1, 12, 17,  1, 17, 17, 17, 10,  3, 10,
       17,  1, 17,  1, 17, 10, 17, 17, 17, 17,  1, 10,  3, 10, 17, 10, 17,
       10, 10, 10, 17, 10,  3, 10, 17,  6, 17,  1, 17, 12, 17, 10,  3, 10,
       17,  1, 17, 13, 17,  1, 17, 10,  3,  6, 17,  6, 17, 17, 17, 17,  1,
       17, 17, 17, 17, 17, 17,  9, 17, 10,  3, 10, 17,  6, 17,  6, 17])

## User Data

In [84]:
location = pd.read_csv("0827.csv")
location.tail()

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1,category2,category3,category4,category5
302,303,121.574140,25.080003,2020-08-27,09:33,5,0.0,-1.0,Taipei Municipal Neihu Vocational High School,Gangqian,聚碩科技,UFC GYM 內科模範館,CTBC Bank,school,subway_station,point_of_interest,gym,bank
303,304,121.573768,25.079590,2020-08-27,09:33,5,0.0,-1.0,Taipei Municipal Neihu Vocational High School,UFC GYM 內科模範館,The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,LOUISA COFFEE 路易莎咖啡 (內湖洲子店),school,gym,real_estate_agency,cafe,cafe
304,305,121.573685,25.079581,2020-08-27,10:38,5,0.0,-1.0,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,聚碩科技,The Executive Centre - Neihu New Century Build...,福來早餐 港墘店,gym,school,point_of_interest,real_estate_agency,restaurant
305,306,121.573685,25.079581,2020-08-27,10:38,5,0.0,-1.0,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,聚碩科技,The Executive Centre - Neihu New Century Build...,福來早餐 港墘店,gym,school,point_of_interest,real_estate_agency,restaurant
306,307,121.573697,25.079611,2020-08-27,10:38,5,0.0,-1.0,UFC GYM 內科模範館,Taipei Municipal Neihu Vocational High School,LOUISA COFFEE 路易莎咖啡 (內湖洲子店),The Executive Centre - Neihu New Century Build...,STARBUCKS COFFEE,gym,school,cafe,real_estate_agency,cafe


In [105]:
stopLoc = location.drop(['category2','category3','category4','category5'], 1)

def string_to_mins(ts):
    a = int(ts.split(':')[0])
    b = int(ts.split(':')[1])
    c = a*60 + b
    return c

#stopLoc['start_time'] = pd.to_datetime(stopLoc['start_time'], format='%H:%M', errors='ignore').apply(string_to_mins)
stopLoc['longitude'] = round(stopLoc['longitude'],3)
stopLoc['latitude'] = round(stopLoc['latitude'],3)
stopLoc

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1
3,4,121.563,25.087,2020-08-26,10:36,4,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,赫里翁 - 接待中心,LOUISA COFFEE 路易莎咖啡 (內湖文湖門市),restaurant
4,5,121.563,25.087,2020-08-26,10:36,4,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,赫里翁 - 接待中心,LOUISA COFFEE 路易莎咖啡 (內湖文湖門市),restaurant
5,6,121.563,25.087,2020-08-26,10:36,4,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,赫里翁 - 接待中心,LOUISA COFFEE 路易莎咖啡 (內湖文湖門市),restaurant
6,7,121.563,25.087,2020-08-26,11:04,4,0.0,-1.000000,軟訊資訊有限公司,阿華食堂,Family Mart,赫里翁 - 接待中心,TOYOTA 內湖營業所,electronics_store
7,8,121.563,25.087,2020-08-26,11:04,4,0.0,-1.000000,軟訊資訊有限公司,阿華食堂,Family Mart,赫里翁 - 接待中心,TOYOTA 內湖營業所,electronics_store
8,9,121.563,25.087,2020-08-26,11:04,4,0.0,-1.000000,軟訊資訊有限公司,阿華食堂,Family Mart,赫里翁 - 接待中心,TOYOTA 內湖營業所,electronics_store
9,10,121.563,25.087,2020-08-26,11:07,4,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,赫里翁 - 接待中心,Takming University of Science and Technology,restaurant


In [106]:
#delete trivial data
dropList = []

for i in range(3):
    for row in range(1,len(stopLoc)-1): 
    
        if stopLoc.iloc[row,1] == stopLoc.iloc[row-1,1]: 
            if stopLoc.iloc[row,2] == stopLoc.iloc[row-1,2]:
                dropList.append(row)
    
        #compared with the last record, over 2 place name duplicated will be deleted
        res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
        if len(res) > 1: 
            dropList.append(row)

        #reset duration
        #stopLoc.iloc[row,6] = stopLoc.iloc[row+1,4] - stopLoc.iloc[row,4] 

        #duration less than 10, kill
        #if stopLoc.iloc[row,6] < 5:
        #    dropList.append(row)

stopLoc = stopLoc.drop(dropList)
stopLoc = stopLoc.reset_index(drop = True)
stopLoc

,location_id,longitude,latitude,start_date,start_time,weekday,duration,speed,name1,name2,name3,name4,name5,category1
0,1,121.576,24.986,2020/07/22,11:00,4,NaN,NaN,myhome,myhome1,myhome2,myhome3,myhome4,c1
1,2,121.443,24.433,2020/07/23,12:00,5,NaN,NaN,apple,apple1,apple2,apple3,apple4,c1
2,3,121.123,24.765,2020/07/24,13:00\n,6,NaN,NaN,banana,banana1,banana2,banana3,banana4,c1
3,4,121.563,25.087,2020-08-26,10:36,4,0.0,-1.000000,阿華食堂,軟訊資訊有限公司,Family Mart,赫里翁 - 接待中心,LOUISA COFFEE 路易莎咖啡 (內湖文湖門市),restaurant
4,13,121.563,25.086,2020-08-26,12:24,4,0.0,-1.000000,TOYOTA 內湖營業所,DUCATI,興沛動物醫院,LOUISA COFFEE 路易莎咖啡 (內湖文湖門市),Family Mart,car_dealer
5,14,25.087,25.087,2020-08-26,12:24,4,0.0,-1.000000,mo家,2,NaN,NaN,NaN,Home
6,16,121.562,25.086,2020-08-26,12:25,4,0.0,1.115088,銀河互動網路股份有限公司,DUCATI,禾伸堂企業股份有限公司,TOYOTA 內湖營業所,興沛動物醫院,school
7,17,25.087,25.087,2020-08-26,12:25,4,0.0,1.115088,mo家,2,NaN,NaN,NaN,Home
8,18,121.561,25.085,2020-08-26,12:26,4,0.0,-1.000000,來來豆漿,MediaTek Inc.,頤園素食小館,7-ELEVEN,全家便利商店 陽湖店,restaurant
9,19,121.562,25.084,2020-08-26,12:28,4,0.0,-1.000000,北勢湖環保公園,來來豆漿,Wen Hu Elementary School,Domino's Pizza,Wen Hu Elementary School,park


In [107]:
#calculate duration
from datetime import datetime
dropList = []
a = 10
for i in range(3):
    for row in range(0,len(stopLoc)-1): 
        
        res = set(stopLoc.iloc[row,8:13]) & set(stopLoc.iloc[row-1,8:13])
        if len(res) > 1: 
            dropList.append(row)

        #reset duration
        #stopLoc.iloc[row,6] = stopLoc.iloc[row+1,4] - stopLoc.iloc[row,4] 

        #duration less than 10, kill
        #if stopLoc.iloc[row,6] < 10:
        #    dropList.append(row)

        #calculate duration
        if stopLoc.iloc[row+1,3] == stopLoc.iloc[row,3]:
            stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')   
        #else:
        #    stopLoc.iloc[row,6] = datetime.strptime(stopLoc.iloc[row+1,4], '%H:%M') + datetime.strptime('24:00') - datetime.strptime(stopLoc.iloc[row,4], '%H:%M')

        if pd.Timedelta(stopLoc.iloc[row,6]) < pd.Timedelta('00:10:00'):
            dropList.append(row)     
            
    stopLoc = stopLoc.drop(dropList)
    stopLoc = stopLoc.reset_index(drop = True)
    a = a + 2
    print(dropList)
    dropList = []

stopLoc = stopLoc.drop(['start_date','name2','name3','name4','name5','speed'], 1)
stopLoc

[4, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 102, 104, 105, 106, 108, 109, 110, 111, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186]
[6, 10]
[]


,location_id,longitude,latitude,start_time,weekday,duration,name1,category1
0,1,121.576,24.986,11:00,4,NaN,myhome,c1
1,2,121.443,24.433,12:00,5,NaN,apple,c1
2,3,121.123,24.765,13:00\n,6,NaN,banana,c1
3,4,121.563,25.087,10:36,4,2:22:00,阿華食堂,restaurant
4,93,121.616,25.058,12:58,4,1:41:00,世貿公園,park
5,99,121.616,25.059,14:39,4,2:46:00,PappaRich金爸爸-南港店,restaurant
6,150,121.545,25.042,17:25,4,0:34:00,源味本鋪古早味現烤蛋糕（忠孝復興店）,bakery
7,160,121.551,25.043,17:59,4,0:12:00,屯京拉麵 敦化南路店,restaurant
8,162,121.552,25.041,18:11,4,1:43:00,Mingyao Department Store,department_store
9,204,121.548,25.043,19:54,4,1:36:00,恩恩努肉飯,restaurant


## Map Category

In [ ]:
stopLoc = pd.merge(stopLoc, loc_to_cat, how='left', on='category1' )
stopLoc = stopLoc.drop(['name1','category1','location_id'],1)
stopLoc['user_id'] = 2
stopLoc = stopLoc[['user_id','weekday','start_time','duration','location']]
stopLoc
#把start_date加回來
#name1,category1拿掉 place_cat對表換成數字

In [ ]:
traing = stopLoc.iloc[:, np.r_[0:4]].values
predictions = clf.predict(traing)

## Myplace

In [ ]:
savedplace = pd.read_csv("savedPlace.csv")
savedplace['place_longitude'] = round(saveplace['place_longitude'],3)
savedplace['place_latitude'] = round(saveplace['place_latitude'],3)
#savedplace = savedplace.loc[savedplace['my_place'] == 1]
savedplace = savedplace.drop(['place_name','my_place','place_id'],1)
savedplace = savedplace.rename(columns={"place_longitude": "longitude", "place_latitude": "latitude"})
savedplace = savedplace.drop_duplicates(subset=['longitude', 'latitude'], keep='last')
savedplace.head()

In [ ]:
stopLoc = pd.merge(stopLoc, savedplace, how='left', on=['longitude','latitude'])
stopLoc